# Random Forests


In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
)
from sklearn.model_selection import KFold, GridSearchCV
import pandas as pd
import numpy as np
import pickle

Read the train and test datasets


In [2]:
train_df = pd.read_csv("Utilities/Training_set.csv")
test_df = pd.read_csv("Utilities/Testing_set.csv")

Convert training set OHE columns to int8


In [3]:
columns_to_convert = train_df.columns[
    17:-1
]  # Columns from 17th column up to but excluding the last column
train_df[columns_to_convert] = train_df[columns_to_convert].astype("int8")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385516 entries, 0 to 385515
Columns: 4073 entries, ScheduledArrTime to DepDelay
dtypes: float64(12), int64(6), int8(4055)
memory usage: 1.5 GB


Convert testing set OHE columns to int8


In [4]:
columns_to_convert = test_df.columns[
    17:-1
]  # Columns from 17th column up to but excluding the last column
test_df[columns_to_convert] = test_df[columns_to_convert].astype("int8")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96379 entries, 0 to 96378
Columns: 4073 entries, ScheduledArrTime to DepDelay
dtypes: float64(12), int64(6), int8(4055)
memory usage: 385.9 MB


Get the training features and label


In [5]:
X_train = train_df.iloc[:, :-1]
y_train = train_df.iloc[:, -1]

Get the testing features and label


In [6]:
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]

Initialize a 5-fold cross-validation object


In [7]:
kf = KFold(n_splits=5)

In [8]:
def bin_label(label: pd.Series):
    label_copy = label.copy()
    label_copy[label <= 30] = 0
    label_copy[label_copy > 30] = 1

    return label_copy

Bin the label


In [9]:
y_train_binned = bin_label(y_train)
y_test_binned = bin_label(y_test)

## Grid Searching


In [11]:
rfc = RandomForestClassifier(n_jobs=5)

In [12]:
param_grid = {
    "max_depth": [None, 30],
    "class_weight": [
        "balanced",
        {0: 10, 1: 1},
    ],
    "n_estimators": [100, 50],
    "min_samples_split": [2, 4],
    "max_features": ["sqrt", "log2"],
}

In [13]:
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=kf, verbose=2)

In [12]:
grid_search.fit(X_train, y_train_binned)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END class_weight=balanced, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time= 2.3min
[CV] END class_weight=balanced, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time= 2.3min
[CV] END class_weight=balanced, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time= 2.3min
[CV] END class_weight=balanced, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time= 2.3min
[CV] END class_weight=balanced, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time= 2.3min
[CV] END class_weight=balanced, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=50; total time= 1.3min
[CV] END class_weight=balanced, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=50; total time= 1.3min
[CV] END class_weight=balanced, max_depth=None, max_features=sqrt

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(n_jobs=5),
             param_grid={'class_weight': ['balanced', {0: 10, 1: 1}],
                         'max_depth': [None, 30],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [2, 4],
                         'n_estimators': [100, 50]},
             verbose=2)

In [13]:
best_params = grid_search.best_params_
best_params["verbose"] = 3
best_params["n_jobs"] = 5
print(best_params)

{'class_weight': 'balanced', 'max_depth': None, 'max_features': 'log2', 'min_samples_split': 4, 'n_estimators': 100, 'verbose': 3, 'n_jobs': 5}


In [14]:
RF_best = RandomForestClassifier(**best_params)

In [15]:
RF_best.fit(X_train, y_train_binned)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100


[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   26.6s


building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69

[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed: 15.4min finished


RandomForestClassifier(class_weight='balanced', max_features='log2',
                       min_samples_split=4, n_jobs=5, verbose=3)

In [19]:
predictions = RF_best.predict(X_test)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    0.8s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    3.4s finished


In [18]:
print(classification_report(y_true=y_test_binned, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.57      0.29      0.38     35520
           1       0.68      0.88      0.76     60859

    accuracy                           0.66     96379
   macro avg       0.63      0.58      0.57     96379
weighted avg       0.64      0.66      0.62     96379



## Additional Tuning


In [13]:
rfc = RandomForestClassifier(
    class_weight={0: 10, 1: 1},
    max_features="sqrt",
    min_samples_split=3,
    max_depth=None,
    n_jobs=5,
    verbose=3,
)

In [14]:
rfc.fit(X_train, y_train_binned)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100


[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   46.3s


building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69

[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:  3.3min finished


RandomForestClassifier(class_weight={0: 10, 1: 1}, min_samples_split=3,
                       n_jobs=5, verbose=3)

In [15]:
predictions = rfc.predict(X_test)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    0.8s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    3.1s finished


In [16]:
print(classification_report(y_true=y_test_binned, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.51      0.50      0.50     35520
           1       0.71      0.72      0.72     60859

    accuracy                           0.64     96379
   macro avg       0.61      0.61      0.61     96379
weighted avg       0.64      0.64      0.64     96379



In [30]:
rfc = RandomForestClassifier(
    class_weight={0: 10, 1: 1},
    max_features="sqrt",
    min_samples_split=3,
    max_depth=100,
    n_jobs=5,
    verbose=3,
)

In [31]:
rfc.fit(X_train, y_train_binned)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100


[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   32.0s


building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70

[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:  2.2min finished


RandomForestClassifier(class_weight={0: 10, 1: 1}, max_depth=100,
                       min_samples_split=3, n_jobs=5, verbose=3)

In [32]:
predictions = rfc.predict(X_test)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    0.4s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    1.5s finished


In [33]:
print(classification_report(y_true=y_test_binned, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.41      0.92      0.56     35520
           1       0.82      0.22      0.34     60859

    accuracy                           0.48     96379
   macro avg       0.62      0.57      0.45     96379
weighted avg       0.67      0.48      0.42     96379



## Final model parameters choice


In [10]:
rfc = RandomForestClassifier(
    class_weight={0: 10, 1: 1},
    max_features="sqrt",
    min_samples_split=3,
    max_depth=None,
    n_jobs=5,
)

Initialize scores lists


In [11]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

Train the model using 5-fold cross-validation


In [13]:
fold = 1

for train, validation in kf.split(X_train):
    print(f"##### FOLD: {fold} #####")
    x_train, x_val = X_train.iloc[train], X_train.iloc[validation]
    y_train, y_val = y_train_binned.iloc[train], y_train_binned.iloc[validation]

    # Fit the model
    rfc.fit(x_train, y_train)

    # Predict on the test set
    predictions = rfc.predict(x_val)

    # Evaluate the model
    precision = precision_score(
        y_true=y_val,
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_val,
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_val, y_pred=predictions)
    f1 = f1_score(
        y_true=y_val,
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

##### FOLD: 1 #####
Precision = 0.6370105159774384
Recall = 0.639396140278066
Accuracy = 0.639396140278066
F1 score = 0.6381106761718084

##### FOLD: 2 #####
Precision = 0.6355794013335666
Recall = 0.637212559822575
Accuracy = 0.637212559822575
F1 score = 0.6363548627699536

##### FOLD: 3 #####
Precision = 0.6340686982618795
Recall = 0.6359804417467544
Accuracy = 0.6359804417467544
F1 score = 0.6349673095339269

##### FOLD: 4 #####
Precision = 0.6339015278499603
Recall = 0.6362528046898305
Accuracy = 0.6362528046898305
F1 score = 0.6349940443922711

##### FOLD: 5 #####
Precision = 0.6334145861567755
Recall = 0.6361360777142264
Accuracy = 0.6361360777142264
F1 score = 0.634658310624199



Print the mean scores of the folds


In [14]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Mean Scores:
Mean Precision = 0.634794945915924
Mean Recall = 0.6369956048502905
Mean Accuracy = 0.6369956048502905
Mean F1 score = 0.6358170406984318


Make predictions on the test set


In [15]:
predictions = rfc.predict(X_test)

Evaluate the model


In [16]:
print(classification_report(y_true=y_test_binned, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.51      0.49      0.50     35520
           1       0.71      0.72      0.72     60859

    accuracy                           0.64     96379
   macro avg       0.61      0.61      0.61     96379
weighted avg       0.63      0.64      0.64     96379



In [18]:
with open("model.pkl", "wb") as file:
    pickle.dump(rfc, file)